In [168]:
import os
import faiss
import json

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [169]:
os.environ["OpenAI_API_KEY"] = os.getenv("GPT_API")

model = ChatOpenAI(model ="gpt-4o")

In [170]:
with open("food_recipe/food_recipes_1-1.json",'r',encoding='utf-8') as file:
    data = json.load(file)

In [171]:
document_recipe = [
    Document(page_content=str(recipe), metadata={"요리명": recipe["요리명"],"요리재료" : recipe["요리재료"], "기본정보" : recipe["기본정보"], "조리순서" : recipe["조리순서"]})
    for recipe in data
]

In [172]:
print(document_recipe)

[Document(metadata={'요리명': '미역국', '요리재료': [{'재료': '미역(마른것)', '용량': '5줌(20g)'}, {'재료': '쇠고기(양지머리)', '용량': '120g'}, {'재료': '물', '용량': '8컵(1'}, {'재료': '재래간장 1과', '용량': '1/2큰술(22ml)'}, {'재료': '마늘(다진 마늘)', '용량': '1큰술(10g)'}, {'재료': '소금', '용량': '작은술(3g)'}, {'재료': '참기름', '용량': '작은술(5ml)'}], '기본정보': {'조리시간': '30분', '분량': '4인분 기준', '칼로리': '81kcal (1인분)'}, '조리순서': ['01. 마른 미역은 찬물에 담가 10분간 불린다. 찬물에 바락바락 씻어 거품이 나오지 않을 때까지 헹군다.', '02. 물기를 꼭 짠 후 적당한 크기로 자른 후 재래간장 1/2큰술을 넣고 조물조물 무친다.', '03. 쇠고기는 한입 크기로 썬 후 달군 냄비에 참기름을 두르고 쇠고기, 마늘을 넣어 볶다가 쇠고기가 거의 익으면 미역을 넣고 볶는다.', '04. 03에 물을 넣고 한소끔 끓인다. 재래간장과 소금으로 간하고 더 끓인다. (물 대신 쌀뜨물을 넣으면 더욱 구수하고 맛있는 미역국을 만들 수 있다.)']}, page_content="{'요리명': '미역국', '요리재료': [{'재료': '미역(마른것)', '용량': '5줌(20g)'}, {'재료': '쇠고기(양지머리)', '용량': '120g'}, {'재료': '물', '용량': '8컵(1'}, {'재료': '재래간장 1과', '용량': '1/2큰술(22ml)'}, {'재료': '마늘(다진 마늘)', '용량': '1큰술(10g)'}, {'재료': '소금', '용량': '작은술(3g)'}, {'재료': '참기름', '용량': '작은술(5ml)'}], '기본정보': {'조리시간': '30분', '분량': '4인분 기준', '칼로리': '81kcal (1인분)'}, '조리순서': [

In [173]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [174]:
index  = faiss.IndexFlatL2(len(embeddings.embed_query("레시피")))

recipe_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [175]:
uuids = [str(uuid4()) for _ in range(len(document_recipe))]

recipe_store.add_documents(documents=document_recipe, ids = uuids)

['88830ca6-e47f-4b7b-a6c7-7403d07c60dd',
 '5a1917cc-ff07-48e6-8d04-aad24ecc76d2',
 '9a4da36b-7fed-4329-a6a7-38a99ad46361',
 'e4705f7e-7fd4-4e2b-9e16-fc0231f7f3e1',
 'ad0fe827-03ba-48c9-a053-bde2909a6657',
 '2fe28c5b-63c3-4ea1-b705-c5759aa3e779',
 '6bdf6a09-d088-4139-9058-28fa0c5814ac',
 '5c70f6de-5140-4b86-a017-4897f70c62c7',
 '5e428a7e-71cb-4822-8072-04a3fb0bc694']

In [176]:
retriever = recipe_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [177]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 주어진 데이터로만 답변을 할 수 있어."),
    ("system", "너는 요리의 전문가야."),
    ("system", "질문에 답변할 때는 항상 데이터를 세밀히 학습하고 정확한 답변을 생성해야 해."),
    ("system", "만약 데이터 안에 유저의 질문이 없다면, 양해를 구하고 관련 데이터를 제공할 수 없음을 알리도록 해."),
    ("system", "질문에 계산이 필요한 경우, 데이터를 기반으로 정확하게 계산하여 결과를 제공해야 합니다."),
    ("system", "다른 데이터와 비교해서 부족한 부분이 잇다면 자동으로 추가해줘 예시로 물이 900mg이상이면 데이터가 추가가 안됬는데 아래의 물 1컵의 요량을 가지고 계산을 해줘."),

    
    ("system", "다음은 답변 형식의 예시야: \n"
               "user: 미역국을 만들고 싶어.\n"
               "ai: 미역국의 재료는 ~입니다.\n"
               "ai: 미역국을 만드는 순서는 다음과 같습니다:\n"
               "ai: 첫 번째 ~~~, 두 번째 ~~~입니다.\n"),
    
    ("system", "또 다른 예시를 들어줄게:\n"
               "user: 내가 만들려고 하는 미역국의 칼로리는 얼마야?\n"
               "ai: 미역국의 칼로리는 1인분당 ~kcal입니다. 이 레시피는 ~인분 기준이므로, 총 칼로리는 ~kcal입니다.\n"),
    
    ("system", "추가 예시:\n"
               "user: 5인분의 레시피로 수정해줘.\n"
               "ai: 현재 미역국 레시피는 ~인분 기준입니다. 5인분으로 수정된 재료는 다음과 같습니다:\n"
               "ai: ~~~.\n"
               "이때도 순서를 전부 알려줘"),
    
    ("user", "다음과 같은 데이터를 학습해:\n{data}"),
    ("user", "그리고 질문에 답해:\n{question}")
])

In [178]:
contextual_prompt = prompt

In [179]:
class DebugPassThrough(RunnablePassthrough):
    
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args,**kwargs)
        return output
    
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config = None, **kwargs):
        data_text = inputs
        
        return {"data": data_text, "question": inputs["question"]}
    
rag_chain_debug = {
    "data" : retriever,
    "question" : DebugPassThrough()
    
} | DebugPassThrough() | ContextToText()| contextual_prompt | model

In [180]:
while True:
    print("--------------------")
    
    query = input("질문을 입력해 주세요 (break 입력시 종료됩니다) : ")
    
    if query.lower() == "break":
        break
    
    response = rag_chain_debug.invoke(query)
    
    print("Question : ", query)
    print(response.content)

--------------------
Question :  미역국 10인분 만들어줘
미역국의 현재 레시피는 4인분 기준입니다. 10인분으로 수정된 재료는 다음과 같습니다:

- 미역(마른것): 12.5줌(50g)
- 쇠고기(양지머리): 300g
- 물: 20컵
- 재래간장: 1과 1/4큰술(55ml)
- 마늘(다진 마늘): 2.5큰술(25g)
- 소금: 7.5g
- 참기름: 12.5ml

조리 순서는 동일합니다:

01. 마른 미역은 찬물에 담가 10분간 불립니다. 찬물에 바락바락 씻어 거품이 나오지 않을 때까지 헹굽니다.
02. 물기를 꼭 짠 후 적당한 크기로 자른 후 재래간장 1/2큰술을 넣고 조물조물 무칩니다.
03. 쇠고기는 한입 크기로 썬 후 달군 냄비에 참기름을 두르고 쇠고기, 마늘을 넣어 볶다가 쇠고기가 거의 익으면 미역을 넣고 볶습니다.
04. 03에 물을 넣고 한소끔 끓입니다. 재래간장과 소금으로 간하고 더 끓입니다. (물 대신 쌀뜨물을 넣으면 더욱 구수하고 맛있는 미역국을 만들 수 있습니다.)
--------------------
